In [ ]:
from io import BytesIO
import requests
import zipfile
import pandas as pd
import pickle
import gensim

In [ ]:
class TaggedLineDocument(object):
    def __init__(self, source):
        self.source = source

    def __iter__(self):
        """Iterate through the lines in the source."""
        with gensim.utils.smart_open(self.source) as fin:
                for line in fin:
                    num, tag, text = gensim.utils.to_unicode(line).split('\t')
                    yield gensim.models.doc2vec.TaggedDocument(text.split(), [tag])

In [ ]:
with open('cpc_output.pickle', 'rb') as f:
    patids = pickle.load(f)
patids = [str(i) for i in patids]

print(len(patids))
patids[10]

In [ ]:
url = 'http://www.patentsview.org/data/20171226/claim.tsv.zip'
response = requests.get(url)

In [ ]:
with zipfile.ZipFile(BytesIO(response.content)) as thezip:
    claim_file = thezip.open('claim.tsv')

In [ ]:
output = pd.DataFrame(columns=['patent_id','text'])

In [ ]:
count = 1
for chunk in pd.read_table(claim_file, delimiter = '\t', chunksize=100000):
    a = (chunk[chunk['patent_id'].isin(patids)][['patent_id','text']])
    if not a.empty:
        output = output.append(a)
        print(str(count)+' out of 920 steps complete!', end='\r', flush=True)
        count += 1

In [ ]:
output.to_csv('biotech_claims.txt', sep='\t')

In [ ]:
documents = TaggedLineDocument('biotech_claims.txt')

In [ ]:
model = doc2vec.Doc2Vec(documents, vector_size=300, window=10, min_count=20, workers=12, sample=1e-5, seed = 42, epochs=10)

In [ ]:
model.save('claim_embedding.doc2vec')